In [1]:
import requests
import time
from json import JSONDecodeError

In [2]:
# API endpoint
url = "https://www.data4library.kr/loanDataL#"

In [3]:
# Parameters
params = {
    "authKey": "f1fc1a713d02a17470687b4edd41210f1fc6568183b647d27bed439dcacd7fd1",   # 실제 발급받은 API 키로 교체해야 합니다.
    "startDt": "20180101",
    "endDt": "20181231",
    "gender": '0',
    "age" : '0',
    "kdc" : '0',
}

In [4]:
# 연도 : 2018 ~ 2022
# 성별 : 여성 남성 미상
# 나이 : 20대 ~ 60대 이상 및 미상
for start_day in ['20180101','20190101','20200101','20210101','20220101']:
    end_day = start_day[0:3] + '1231'
    params['startDt'] = start_day
    params['endDt'] = end_day
    for gender in [0,1,2]:
        params['gender'] = str(gender)
        for age in [20,30,40,50,60,-1]:
            params['age'] = str(age)
            response = requests.get(url, params=params)
            time.sleep(5)
            
            if response.status_code == 200:
                try:
                    data = response.json()
                    print(data)
                except JSONDecodeError:
                    print("Failed to decode JSON from response.")
                    print(response.text)
            else:
                print("Error:", response.status_code)
# 쥐피티에 나와 있는 내용 보고 오류 수정하기

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

...

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

Failed to decode JSON from response.






<!DOCTYPE html>
<html lang="ko">
<head>
	<meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="Content-Security-Policy" content="default-src *; style-src 'self' 'unsafe-inline' https://www.gstatic.com; script-src 'self'
'unsafe-inline' 'unsafe-eval' https://www.googletagmanager.com http://www.google.com https://www.googletagmanager.com https://www.gstatic.com https://ds.nl.go.kr https://sgisapi.kostat.go.kr">

    <!-- 현재페이지 제목 - 첫번째 하위메뉴 - 메인메뉴 -->
	<title>
	
	
	
	
	인기대출도서 - 데이터 제공 -
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	도서관 정보나루
	</title>
	<link rel="shortcut icon" href="/resources/img/favicon.ico" type="image/x-icon" />
    <link rel="icon" href="/resources/img/favicon.ico" type="image/x-icon" />
	<link rel="stylesheet" type="text/css" href="/resources/css/

KeyboardInterrupt: 